In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import pygad
import yfinance as yf
from tqdm import tqdm

In [2]:
train = yf.download("META", start="2015-01-01", end="2020-01-01").reset_index()

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
train

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,78.580002,78.930000,77.699997,78.449997,78.366852,18177500
1,2015-01-05,77.980003,79.250000,76.860001,77.190002,77.108192,26452200
2,2015-01-06,77.230003,77.589996,75.360001,76.150002,76.069290,27399300
3,2015-01-07,76.760002,77.360001,75.820000,76.150002,76.069290,22045300
4,2015-01-08,76.739998,78.230003,76.080002,78.180000,78.097137,23961000
...,...,...,...,...,...,...,...
1253,2019-12-24,206.300003,206.789993,205.000000,205.119995,204.902588,6046300
1254,2019-12-26,205.570007,207.820007,205.309998,207.789993,207.569763,9350700
1255,2019-12-27,208.669998,208.929993,206.589996,208.100006,207.879440,10284200
1256,2019-12-30,207.860001,207.899994,203.899994,204.410004,204.193344,10524300


In [4]:
def fitness_func(self, solution, sol_idx):

    # Get Reward from train data
    reward, wins, losses, pnl = get_result(
        train, solution[0], solution[1], solution[2], solution[3]
    )

    # Return the solution reward
    return reward

In [5]:
def get_result(df, buy_length, buy_std, sell_length, sell_std, is_test=False):

    # Round to 2 digit to avoid the Bollinger bands function to generate weird field names
    buy_std = round(buy_std, 2)
    sell_std = round(sell_std, 2)

    # Generate suffixes for Bollinger bands fields
    buy_suffix = f"{int(buy_length)}_{buy_std}"
    sell_suffix = f"{int(sell_length)}_{sell_std}"

    # Generate a copy to avoid changing the original data
    df = df.copy().reset_index(drop=True)

    # Calculate Bollinger bands based on parameters
    if not f"BBU_{buy_suffix}" in df.columns:
        df.ta.bbands(Close=df["Close"], length=buy_length, std=buy_std, append=True)
    if not f"BBU_{sell_suffix}" in df.columns:
        df.ta.bbands(Close=df["Close"], length=sell_length, std=sell_std, append=True)
    df = df.dropna()

    # Buy Signal
    df["signal"] = np.where(df["Close"] < df[f"BBL_{buy_suffix}"], 1, 0)

    # Sell Signal
    df["signal"] = np.where(df["Close"] > df[f"BBU_{sell_suffix}"], -1, df["signal"])

    # Remove all rows without operations, rows with the same consecutive operation, first row selling, and last row buying
    result = df[df["signal"] != 0]
    result = result[result["signal"] != result["signal"].shift()]
    if (len(result) > 0) and (result.iat[0, -1] == -1):
        result = result.iloc[1:]
    if (len(result) > 0) and (result.iat[-1, -1] == 1):
        result = result.iloc[:-1]

    # Calculate the reward & result / operation
    result["reward"] = np.where(
        result["signal"] == -1,
        (result["Close"] - result["Close"].shift()) * (CASH // result["Close"].shift()),
        0,
    )
    result["wins"] = np.where(result["reward"] > 0, 1, 0)
    result["losses"] = np.where(result["reward"] < 0, 1, 0)

    # Generate window and filter windows without operations
    result_window = (
        result.set_index("Date")
        .resample("3M")
        .agg({"Close": "last", "reward": "sum", "wins": "sum", "losses": "sum"})
        .reset_index()
    )

    min_operations = 252  # 1 Year
    result_window = result_window[
        (result_window["wins"] + result_window["losses"]) != 0
    ]

    # Generate the result
    wins = result_window["wins"].mean() if len(result_window) > 0 else 0
    losses = result_window["losses"].mean() if len(result_window) > 0 else 0
    reward = (
        result_window["reward"].mean()
        if (min_operations < (wins + losses)) or is_test
        else -min_operations + (wins + losses)
    )
    pnl = result_window["reward"].sum()

    return reward, wins, losses, pnl

In [6]:
CASH = 10000
SOLUTIONS = 100
GENERATIONS = 50

In [7]:
with tqdm(total=GENERATIONS) as pbar:

    ga_instance = pygad.GA(
        num_generations=GENERATIONS,
        num_parents_mating=5,
        fitness_func=fitness_func,
        sol_per_pop=SOLUTIONS,
        num_genes=4,
        gene_space=[
            {"low": 1, "high": 200, "step": 1},
            {"low": 0.1, "high": 3, "step": 0.01},
            {"low": 1, "high": 200, "step": 1},
            {"low": 0.1, "high": 3, "step": 0.01},
        ],
        parent_selection_type="sss",
        crossover_type="single_point",
        mutation_type="random",
        mutation_num_genes=1,
        keep_parents=-1,
        random_seed=42,
        on_generation=lambda _: pbar.update(1),
    )

    ga_instance.run()

  0%|          | 0/50 [00:00<?, ?it/s]/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/var/tmp/ipykernel_1363257/4121374385.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["signal"] = np.wh

KeyboardInterrupt: 

In [ ]:
# Show details of the best solution.
solution, solution_fitness, _ = ga_instance.best_solution()

print(f" Best Solution Parameters ".center(60, "*"))
print(f"Buy Length    : {solution[0]:.0f}")
print(f"Buy Std       : {solution[1]:.2f}")
print(f"Sell Length   : {solution[2]:.0f}")
print(f"Sell Std      : {solution[3]:.2f}")

# Get result from train data
reward, wins, losses, pnl = get_result(
    train, solution[0], solution[1], solution[2], solution[3]
)

# Show the train result
print(f" Result (TRAIN) ".center(60, "*"))
print(f"* Reward                   : {reward:.2f}")
print(
    f'* Profit / Loss (B&H)      : {(train["Close"].iloc[-1] - train["Close"].iloc[0]) * (CASH // train["Close"].iloc[0]):.2f}'
)
print(f"* Profit / Loss (Strategy) : {pnl:.2f}")
print(f"* Wins / Losses            : {wins:.2f} / {losses:.2f}")
print(
    f"* Win Rate                 : {(100 * (wins/(wins + losses)) if wins + losses > 0 else 0):.2f}%"
)

# Get result from test data